<a href="https://colab.research.google.com/github/CorinneBean/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http:

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-07 16:32:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.88MB/s    in 0.2s    

2022-09-07 16:32:58 (4.88 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

2643619

Perform analysis on the vine reviews

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create the vine_table. DataFrame
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [ ]:
vine_table = vine_table.dropna()
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [ ]:
rows = vine_table.count()
print({rows})

{2643614}


In [ ]:
#Filter data to create dataframe where there 
#are 20 or more total votes
total_votes_df=vine_table.filter("total_votes>=20")
total_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [ ]:
rows = total_votes_df.count()
print({rows})

{39376}


In [ ]:
#Filter data to create a dataframe where % of 
#helpful votes is equal => 50%
percent_votes_df =total_votes_df.filter("helpful_votes/total_votes>=0.50")
percent_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [ ]:
rows = percent_votes_df.count()
print({rows})

{38010}


In [ ]:
#Create a data frame where there is a vine review
vine_review_df = percent_votes_df.filter(percent_votes_df["vine"]=="Y")
vine_review_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|         36|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|     

In [ ]:
#Create a data frame where there is a vine review
non_vine_review_df = percent_votes_df.filter(percent_votes_df["vine"]=="N")
non_vine_review_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [ ]:
#Determine the number of review, number of 5 star reviews, and percentage of 5 star for vine and non-vine
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = percent_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|          170|                  65| 38.23529411764706|
|   N|        37840|               20612|54.471458773784356|
+----+-------------+--------------------+------------------+



In [ ]:
#Additional Analysis
#Average Star Rating
from pyspark.sql.functions import avg
vine_review_df.select(avg("star_rating")).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.929411764705882|
+-----------------+



In [ ]:
non_vine_review_df.select(avg("star_rating")).show()

+------------------+
|  avg(star_rating)|
+------------------+
|3.7647463002114163|
+------------------+

